# Day 9

## part 1

- digits alternate representing:
    - files
    - free space
- File ID starts at 0 and increments each block
- to compress the files - move them one at a time from the lend of the disk into free space at the beginning.
- the checksum is the sum of the products of each file id and block index

In [1]:
from copy import deepcopy
from dataclasses import dataclass
import logging

from tqdm import tqdm

from advent_of_code_utils.advent_of_code_utils import (
    parse_from_file, ParseConfig as PC, markdown
)

log = logging.getLogger('day 9')
logging.basicConfig(level=logging.INFO)

In [2]:
# lets work on the example first
example = '2333133121414131402'
ex_disk = [int(char) for char in example]

@dataclass
class FileBlocks:
    id: int
    start: int
    end: int

    @property
    def len(self) -> int:
        return self.end - self.start

    def range(self):
        return range(self.start, self.end)

@dataclass
class Disk:
    files: list[FileBlocks]

    @property
    def len(self) -> int:
        return len(self.files)
    
    @property
    def debug_str(self) -> str:
        """returns a string like the example for debugging"""
        output = ['.']*self.files[-1].end
        previous = None
        for fileblock in self.files:
            for index in fileblock.range():
                output[index] = f'{fileblock.id}'
        return ''.join(output)

    def get_next_gap(self) -> FileBlocks | None:
        """
        returns a fileblock starting and ending at the next gap or None if no 
        gaps are found

        uses the id field as the index where the gap is
        """
        for index, (f1, f2) in \
                enumerate(zip(self.files[:-1], self.files[1:]), start=1):
            if f1.end != f2.start:
                log.debug(f'gap found: [{f1.end} -> {f2.start})')
                return FileBlocks(index, f1.end, f2.start)
        else:
            return None

    def defrag(self) -> None:
        """defragments the disk"""
        gap = self.get_next_gap()
        while gap is not None:
            log.debug(f'filling gap: {gap}')
            block = self.files.pop()
            log.debug(f'Moving {block}')
            # if the block just fits in the gap
            if block.len <= gap.len:
                block_len = block.len
                block.start = gap.start
                block.end = block.start + block_len
                self.files.insert(gap.id, block)
            # else put as much block in as we can and pop the rest on the end
            else:
                gap_index = gap.id  # grab the gap index
                gap.id = block.id  # then just use the gap as a block
                block.end -= gap.len  # then eat the gap len off the block
                self.files.insert(gap_index, gap)  # pop in the gap
                self.files.append(block)  # and reappend the shortended block
            gap = self.get_next_gap()
            if log.level <= logging.DEBUG:
                log.debug(f'{self.debug_str}')
    
    @property
    def checksum(self) -> int:
        """finds the sum of the product of each file id and index"""
        total = 0
        index = 0
        for block in self.files:
            for _ in range(block.len):
                total += index * block.id
                index += 1
        return total

def defrag_input(input_disk: list[int]) -> Disk:
    """returns the defragmented disk"""
    # first convert to a Disk type object
    disk = Disk([])
    is_file = True
    start = 0
    file_id = 0
    for digit in input_disk:
        if is_file:
            new_block = FileBlocks(file_id, start, start + digit)
            disk.files.append(new_block)
            log.debug(f'added {new_block=}')
            file_id += 1
        else:
            log.debug(f'Incremented start by {digit}')
        start += digit
        is_file = not is_file
    log.info(f'Mapped disk with {disk.len=}')

    # next defragment the disk
    disk.defrag()

    return disk

log.setLevel(logging.DEBUG)
ex_defrag = defrag_input(ex_disk)
log.info(f'{ex_defrag.checksum=}')

DEBUG:day 9:added new_block=FileBlocks(id=0, start=0, end=2)
DEBUG:day 9:Incremented start by 3
DEBUG:day 9:added new_block=FileBlocks(id=1, start=5, end=8)
DEBUG:day 9:Incremented start by 3
DEBUG:day 9:added new_block=FileBlocks(id=2, start=11, end=12)
DEBUG:day 9:Incremented start by 3
DEBUG:day 9:added new_block=FileBlocks(id=3, start=15, end=18)
DEBUG:day 9:Incremented start by 1
DEBUG:day 9:added new_block=FileBlocks(id=4, start=19, end=21)
DEBUG:day 9:Incremented start by 1
DEBUG:day 9:added new_block=FileBlocks(id=5, start=22, end=26)
DEBUG:day 9:Incremented start by 1
DEBUG:day 9:added new_block=FileBlocks(id=6, start=27, end=31)
DEBUG:day 9:Incremented start by 1
DEBUG:day 9:added new_block=FileBlocks(id=7, start=32, end=35)
DEBUG:day 9:Incremented start by 1
DEBUG:day 9:added new_block=FileBlocks(id=8, start=36, end=40)
DEBUG:day 9:Incremented start by 0
DEBUG:day 9:added new_block=FileBlocks(id=9, start=40, end=42)
INFO:day 9:Mapped disk with disk.len=10
DEBUG:day 9:gap fou

In [3]:
# great that works! let's solve the real one
parser = PC('', int)
input_disk = parse_from_file('day_9.txt', parser)
log.setLevel(logging.INFO)
disk = defrag_input(input_disk)
checksum = disk.checksum
markdown(f'The defragmented disk checksum is: {checksum}')

INFO:advent_of_code_utils.py:19999 items loaded from "day_9.txt"
INFO:day 9:Mapped disk with disk.len=10000


The defragmented disk checksum is: 6463499258318

## part 2
- turns out what I was doing was fragmentation not defragmentation!
- now instead try to fit the blocks in without splitting them
- only try move each block once, else keep it where it was

In [24]:
@dataclass
class SuperBlock:
    id: int
    start: int
    end: int
    tried: bool = False

    @property
    def len(self) -> int:
        return self.end - self.start

    def range(self):
        return range(self.start, self.end)

    @property
    def checksum(self) -> int:
        return self.id * (sum(self.range()))

class SuperDisk(Disk):
    """like a disk but super!"""
    def parse_input(self, input_disk: list) -> None:
        """overwrites the files with the input provided"""
        self.files = []
        is_file = True
        start = 0
        file_id = 0
        for digit in input_disk:
            if is_file:
                new_block = SuperBlock(file_id, start, start + digit)
                self.files.append(new_block)
                log.debug(f'added {new_block=}')
                file_id += 1
            else:
                log.debug(f'Incremented start by {digit}')
            start += digit
            is_file = not is_file
        log.info(f'Mapped disk with {self.len} files')
    
    def get_next_to_try(self) -> int | None:
        """
        returns the index of the next block to try compacting or None if all
        have been tried
        """
        for rev_index, block in enumerate(reversed(self.files)):
            if not block.tried:
                index = self.len - rev_index - 1 
                log.debug(f'next index to try: {index}')
                return index
        else:
            return None
    
    def get_useful_gap(self, block: SuperBlock) -> FileBlocks | None:
        """
        returns the next gap where the block would fit or None if it can't fit
        anywhere closer
        """
        for index, (f1, f2) in \
                enumerate(zip(self.files[:-1], self.files[1:]), start=1):
            if f2.start > block.start:
                log.debug(f'reached block without finding gap it can fit in')
                return None
            elif f2.start - f1.end >= block.len:
                log.debug('gap found!')
                return FileBlocks(index, f1.end, f2.start)
            # this one catches if we can just shift the block up a bit
            # elif f1.end <= block.start <= f2.start:
            #     log.debug('block shift gap found!')
            #     return FileBlocks(index, f1.end, f2.start)

    def compact(self) -> None:
        """compacts this superdisk"""
        log.info('Compacting disk...')
        next_index = self.get_next_to_try()
        while next_index is not None:
            # block = self.files.pop(next_index)
            block = self.files[next_index]
            log.debug(f'Trying {block}')
            gap = self.get_useful_gap(block)
            block.tried = True
            self.files.pop(next_index)
            if gap is None:
                self.files.insert(next_index, block)
                log.debug(f'block can\'t be moved')
            else:
                log.debug(f'filling gap: {gap}')
                block_len = block.len
                block.start = gap.start
                block.end = block.start + block_len
                self.files.insert(gap.id, block)

            next_index = self.get_next_to_try()
            if log.level <= logging.DEBUG:
                log.debug(self.debug_str)
        
    @property
    def checksum(self) -> int:
        """finds the checksum of the superdisk"""
        return sum((file.checksum for file in self.files))


log.setLevel(logging.DEBUG)
sd = SuperDisk([])
sd.parse_input(ex_disk)        
sd.compact()
log.info(f'{sd.checksum=}')

DEBUG:day 9:added new_block=SuperBlock(id=0, start=0, end=2, tried=False)
DEBUG:day 9:Incremented start by 3
DEBUG:day 9:added new_block=SuperBlock(id=1, start=5, end=8, tried=False)
DEBUG:day 9:Incremented start by 3
DEBUG:day 9:added new_block=SuperBlock(id=2, start=11, end=12, tried=False)
DEBUG:day 9:Incremented start by 3
DEBUG:day 9:added new_block=SuperBlock(id=3, start=15, end=18, tried=False)
DEBUG:day 9:Incremented start by 1
DEBUG:day 9:added new_block=SuperBlock(id=4, start=19, end=21, tried=False)
DEBUG:day 9:Incremented start by 1
DEBUG:day 9:added new_block=SuperBlock(id=5, start=22, end=26, tried=False)
DEBUG:day 9:Incremented start by 1
DEBUG:day 9:added new_block=SuperBlock(id=6, start=27, end=31, tried=False)
DEBUG:day 9:Incremented start by 1
DEBUG:day 9:added new_block=SuperBlock(id=7, start=32, end=35, tried=False)
DEBUG:day 9:Incremented start by 1
DEBUG:day 9:added new_block=SuperBlock(id=8, start=36, end=40, tried=False)
DEBUG:day 9:Incremented start by 0
DEBUG

At this point, despite getting the right answer for the example, I kept getting an answer that was too high for the main puzzle.

It turns out the example isn't exhaustive of all the cases you can face.

It is apparently valid to move a block into a gap that fits, right in front of it, if the gap is big enough.

`00..11` -> `0011..`

I originally thought any gap would be valid in this case since technically the only thing blocking the `1` file is itself but this is not valid and gives the wrong answer for the example.

`00.11` -x-> `0011.`

In [25]:
test_input = [int(v) for v in '222']  # this produces '00..11'
log.setLevel(logging.DEBUG)
sd = SuperDisk([])
sd.parse_input(test_input)
sd.compact()

DEBUG:day 9:added new_block=SuperBlock(id=0, start=0, end=2, tried=False)
DEBUG:day 9:Incremented start by 2
DEBUG:day 9:added new_block=SuperBlock(id=1, start=4, end=6, tried=False)
INFO:day 9:Mapped disk with 2 files
INFO:day 9:Compacting disk...
DEBUG:day 9:next index to try: 1
DEBUG:day 9:Trying SuperBlock(id=1, start=4, end=6, tried=False)
DEBUG:day 9:gap found!
DEBUG:day 9:filling gap: FileBlocks(id=1, start=2, end=4)
DEBUG:day 9:next index to try: 0
DEBUG:day 9:0011
DEBUG:day 9:Trying SuperBlock(id=0, start=0, end=2, tried=False)
DEBUG:day 9:reached block without finding gap it can fit in
DEBUG:day 9:block can't be moved
DEBUG:day 9:0011


In [22]:
# great that works, let's try solving!
log.setLevel(logging.INFO)
super_disk = SuperDisk([])
super_disk.parse_input(input_disk)
super_disk.compact()

INFO:day 9:Mapped disk with 10000 files
INFO:day 9:Compacting disk...


In [23]:
checksum = super_disk.checksum
markdown(f'The defragmented disk checksum is: {checksum}')

The defragmented disk checksum is: 6493634986625